In [ ]:
# Start writing code here...
# Imports
import pandas as pd
import numpy as np
import glob

import time

import sklearn
from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier


set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24



Pandas   1.2.5
Sklearn  0.24.2


In [ ]:
import os

Dataset is delimited as CSV values as follows;

X ; Y; Z; Pressure; GripAngle; Timestamp; Test ID

----------------
Test ID: <br>
0: Static Spiral Test ( Draw on the given spiral pattern)<br>
1: Dynamic Spiral Test ( Spiral pattern will blink in a certain time, so subjects need to continue on their draw)<br>
2: Circular Motion Test (Subjectd draw circles around the red point)

In [ ]:
col = ["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID","Class"]
control_data = pd.DataFrame()
control_file = glob.glob("/work/parkinson/control/*.txt")

for f in control_file:
    data = pd.read_csv(f, delimiter=";")
    data.columns = col
    # print(data)
    control_data = pd.concat([control_data, data], axis=0)

print(control_data.shape)

#creating the class column
control_data["Class"] = 0

control_data



ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [ ]:
 col = ["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID"]
parkinson_data = pd.DataFrame()
parkinson_file = glob.glob("/work/parkinson/parkinson/*.txt")

for f in parkinson_file:
    data = pd.read_csv(f, delimiter=";")
    data.columns = col
    # print(data)
    parkinson_data = pd.concat([parkinson_data, data], axis=0)

print(parkinson_data.shape)

#creating the class column
parkinson_data["Class"] = 1



(404371, 7)


X            0
Y            0
Z            0
Pressure     0
GripAngle    0
Timestamp    0
Test ID      0
Class        0
dtype: int64

In [ ]:
df = pd.concat([parkinson_data, control_data], axis=0)
df

extra_sample = df.sample(df.shape[0] // 3 )

df_extra = pd.concat([df, extra_sample], axis=0)

In [ ]:
print(df)
print(df_extra)

         X    Y  Z  Pressure  GripAngle  Timestamp  Test ID  Class
0      201  200  0        86        900   68140154        0      1
1      201  200  0       123        900   68140161        0      1
2      201  200  0       146        900   68140168        0      1
3      201  200  0       158        900   68140175        0      1
4      201  200  0       159        900   68140182        0      1
...    ...  ... ..       ...        ...        ...      ...    ...
13231  400  197  0       632       1610    3756201        1      0
13232  400  197  0       624       1610    3756208        1      0
13233  400  197  0       594       1620    3756215        1      0
13234  400  198  0       543       1610    3756222        1      0
13235  400  198  0       406       1620    3756229        1      0

[524666 rows x 8 columns]
        X    Y    Z  Pressure  GripAngle  Timestamp  Test ID  Class
0     201  200    0        86        900   68140154        0      1
1     201  200    0       123    

In [ ]:
std_pressure = df_extra['Pressure'].std()/10

df_extra['Enh_pressure'] = df_extra['Pressure'] + std_pressure
# Health and sick will hget same
# Presseure std for helthy 
# Pressure and grip angle to create new column

In [ ]:
# Preprocessing 
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
normalizer = PowerTransformer()

In [ ]:
df_scaled = scaler.fit_transform(df_extra)

In [ ]:
df_scaled.shape

(699554, 9)

In [ ]:
df_norm = normalizer.fit_transform(df_scaled)
df_norm.shape

(699554, 9)

In [ ]:
df_norm

array([[-0.20157037, -0.31102313, -0.33335057, ..., -1.07867443,
         0.54540759, -1.67736351],
       [-0.20157037, -0.31102313, -0.33335057, ..., -1.07867443,
         0.54540759, -1.60618066],
       [-0.20157037, -0.31102313, -0.33335057, ..., -1.07867443,
         0.54540759, -1.56080826],
       ...,
       [ 0.19467193,  0.19909313,  3.29096956, ...,  1.46049356,
        -1.83349118, -1.83500277],
       [-1.29905721,  1.5879722 , -0.33335057, ...,  0.40499388,
         0.54540759,  1.4612787 ],
       [-0.17052468, -0.43145119, -0.33335057, ..., -1.07867443,
        -1.83349118, -0.29252996]])

In [ ]:
# Append 0 or 1 to class
# Create sequence for healthy and not healthy
# We should go file by file

In [ ]:
 def pairing(df_norm, seq_len =6):  #143*15 = 15s 
     
     for i in range(0, (df_norm.shape[0]-1) // seq_len, seq_len + 1): #creating for loop for the sequence
         
         seq = np.zeros((seq_len, df_norm.shape[1])) 
         for j in range(seq_len):  #iterating through the sequence
            
            seq[j] = data[i+j]
         x.append(seq)
         y.append(df_norm[i+seq_len,1])  # append 
     return np.array(x), np.array(y)
 x, y = pairing(df_norm)


print(df_norm.shape)
print(x.shape)
print(y.shape)

KeyError: 0

In [ ]:
# 1.column Var,
# 2. mean
for f in :

    data = pd.read_csv(f, delimiter=";")
    data.columns = col
    for col in os.listdir('parkinson'+'/'+'control'+'/'+f): 
        print(col)
    # print(data)
    # control_data = pd.concat([control_data, data], axis=0)


SyntaxError: invalid syntax (1515986256.py, line 3)

In [ ]:
col = ["X", "Y", "Z", "Pressure", "GripAngle", "Timestamp", "Test ID", "Class"]
control_data = pd.DataFrame()
control_file = glob.glob("/work/parkinson/control/*.txt")

df_listC = []
df_listP = []
# "/work/parkinson/parkinson/*.txt" -parkinson
for f in control_file:
    data = pd.read_csv(f, delimiter=";")
    data["Class"] = 0
    data.columns = col
    df_listC.append(data)


parkinson_data = pd.DataFrame()
parkinson_file = glob.glob("/work/parkinson/parkinson/*.txt")


for f in parkinson_file:
    data = pd.read_csv(f, delimiter=";")
    data["Class"] = 0
    data.columns = col
    df_listP.append(data)


print(len(df_listC))
print(len(df_listP))

15
62


In [ ]:
# df_list[]

SyntaxError: invalid syntax (4185473471.py, line 4)

In [ ]:
# for f in df:
#     print(df[f])
        

In [ ]:
len(df_listP)

62

In [ ]:
type(df_listP)

list

In [ ]:
df_listP[0]

,X,Y,Z,Pressure,GripAngle,Timestamp,Test ID,Class
0,201,200,0,86,900,68140154,0,0
1,201,200,0,123,900,68140161,0,0
2,201,200,0,146,900,68140168,0,0
3,201,200,0,158,900,68140175,0,0
4,201,200,0,159,900,68140182,0,0
...,...,...,...,...,...,...,...,...
9538,401,207,0,267,1510,68223756,1,0
9539,401,206,0,207,1550,68223763,1,0
9540,401,205,0,145,1550,68223770,1,0
9541,400,204,0,62,1580,68223777,1,0


In [ ]:
variance_X = []
variance_Y = []
std_Z  = []
mean_pressure = []
mean_grip = []
std_time = []
for f in df_listP:
    # print(type(f))
    # print(f.columns)

    # print(f['X'].var())
    variance_X.append(f['X'].var())
    variance_Y.append(f['Y'].var())
    std_Z.append(f['Z'].std())
    mean_pressure.append(f['Pressure'].mean())
    mean_grip.append(f['GripAngle'].mean())
    std_time.append(f['Timestamp'].std())



In [ ]:
variance_X

[8914.3725716952,
 9200.117578875006,
 36173.10064548719,
 6822.843078080139,
 6097.499737708577,
 7474.5290120580585,
 8057.943004606125,
 13362.962480542368,
 11801.367594625788,
 8075.881249002759,
 7197.480523166511,
 6934.414602937071,
 7503.5769279057395,
 4931.574502012839,
 5805.373944124754,
 6777.5476056175885,
 9411.77022299954,
 4399.6185915402375,
 8026.748518990466,
 8978.42966326517,
 7692.702004603672,
 6780.820305007627,
 9330.435227408721,
 7446.508226118581,
 27065.295326728316,
 7356.849016775852,
 27938.30489876121,
 9244.19744029345,
 14895.6834713766,
 7468.220841559246,
 8929.290774299341,
 5425.35091032504,
 30223.638743584055,
 5915.752266730526,
 31416.945058360092,
 6589.2279562178,
 7028.266279472293,
 8370.690715964647,
 10199.17334289208,
 4357.403537505567,
 4750.678737589487,
 12685.228383619276,
 6630.398451530456,
 8287.752645473749,
 7064.544136048769,
 8588.119650859255,
 10129.300665004703,
 6851.444962082069,
 16440.992024331987,
 11845.8546512858

In [ ]:
#########################################################################################################################################################

In [ ]:
df_listP[0].columns

Index(['X', 'Y', 'Z', 'Pressure', 'GripAngle', 'Timestamp', 'Test ID',
       'Class'],
      dtype='object')

In [ ]:
len(variance_X)

62

In [ ]:
mean_grip

[1158.712145027769,
 1288.915609969392,
 1392.8911362284355,
 1481.8606304811567,
 1324.48547979798,
 1589.2831397677212,
 2228.9687086822387,
 1452.0177439797212,
 2004.6099751163042,
 1254.2721411605023,
 1138.3673051316193,
 1222.816018799569,
 1886.8268173967656,
 1581.1919268849963,
 756.456561922366,
 657.073835125448,
 1329.6434985632184,
 1213.1156893819334,
 2308.9362962962964,
 999.0010807472595,
 1317.198565665621,
 897.2846567967698,
 1678.7748149888405,
 1072.9494314057586,
 1527.5057736720555,
 927.9845261121857,
 1117.2005842259007,
 1276.791294642857,
 915.4705472289996,
 2632.271142909558,
 981.0237684238481,
 1028.0374357891774,
 1366.2397820163487,
 849.8974836905871,
 1481.7156659765355,
 1150.0802878076602,
 977.2892679459844,
 652.1148697980044,
 981.2572701398342,
 1439.735460844006,
 555.3623649078195,
 587.8670807453416,
 1455.87178241865,
 1023.8019682667202,
 1132.8785788202829,
 1571.2426937738246,
 1413.610361575823,
 2250.6091370558374,
 1269.9078341013824

In [ ]:
variance_X = []
variance_Y = []
std_Z  = []
mean_pressure = []
mean_grip = []
std_time = []
def pairing(df_listP, seq_len =60):  #143*15 = 15s 
     
      #creating for loop for the sequence
     for f in df_listP:
       
        variance_X.append(f['X'].var())
        variance_Y.append(f['Y'].var())
        std_Z.append(f['Z'].std())
        mean_pressure.append(f['Pressure'].mean())
        mean_grip.append(f['GripAngle'].mean())
        std_time.append(f['Timestamp'].std())



In [ ]:
# df_listP[0].columns

In [ ]:
variance_X = []
variance_Y = []
std_Z  = []
mean_pressure = []
mean_grip = []
std_time = []
def pairing_C(df_listC, seq_len =60):  #143*15 = 15s 
     
      #creating for loop for the sequence
     for f in df_listC:
       
        variance_X.append(f['X'].var())
        variance_Y.append(f['Y'].var())
        std_Z.append(f['Z'].std())
        mean_pressure.append(f['Pressure'].mean())
        mean_grip.append(f['GripAngle'].mean())
        std_time.append(f['Timestamp'].std())
      return variance_X,variance_Y,std_Z,mean_pressure,mean_grip,std_time
def pairing_P(df_listP, seq_len =60):  #143*15 = 15s 
     
      #creating for loop for the sequence
      for f in df_listP:
       
        variance_X.append(f['X'].var())
        variance_Y.append(f['Y'].var())
        std_Z.append(f['Z'].std())
        mean_pressure.append(f['Pressure'].mean())
        mean_grip.append(f['GripAngle'].mean())
        std_time.append(f['Timestamp'].std())
      return variance_X,variance_Y,std_Z,mean_pressure,mean_grip,std_time

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [ ]:
variance_X

[]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfc = RandomForestRegressor()

In [ ]:
model = rfc.fit(X_train_d2,y_train)

In [ ]:
pred = model.predict(X_test_d2)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=94e431f8-42fe-427d-ad9d-8b943b5f884e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>